## Product attribute Extraction



Library

In [ ]:
import tensorflow as tf
#from tensorflow.keras.layers import Input, Dense, Embedding, Flatten 
from tensorflow.keras.models import Model
#from tensorflow.keras.optimizers import SGD
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *
from keras.initializers import Constant
import keras.backend as K
from tensorflow.keras import models

Dataset

In [ ]:
dataset = pd.read_csv('Train.csv')


Dataset Visualization

In [ ]:
dataset.head()

Taking the important features of the dataset for further processing

In [ ]:
dataset= dataset[['title','brand']]
dataset.head()

Tokenizer

In [ ]:
featurestoCare = 20000
MAX_SEQUENCE_LENGTH=50
tokens = Tokenizer(num_words=featurestoCare)
tokens.fit_on_texts(dataset['title'].values)
x = tokens.texts_to_sequences(dataset['title'].values)
x = pad_sequences(x)
tokens.fit_on_texts(dataset['brand'].values)
y = tokens.texts_to_sequences(dataset['brand'].values)
y = pad_sequences(y)
y=y[:,2:]

In [ ]:
y[10]

Train Test Splitting

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1)


Word Embedding

In [ ]:
word_index = tokens.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
# Word Embedding 
embedding_dim = 1000
num_words = min(featurestoCare, len(word_index)) + 1
# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))
# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector

In [ ]:
# Embedding Layer
embedding_layer=Embedding(num_words,
                    embedding_dim,
                    embeddings_initializer=Constant(embedding_matrix),
                    trainable=True)

CNN Model

In [ ]:
# CNN Model 
sequence_input = Input(shape=(21,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

#Convolution Layer
x = Conv1D(128, 3, activation='relu')(embedded_sequences)

x = Conv1D(64, 3, activation='relu')(x)

x = Conv1D(64, 3, activation='relu')(x)
x = Conv1D(64, 3, activation='relu')(x)

x = Flatten()(x)

#Dense Layer
x = Dense(64, activation='relu')(x)
preds = Dense(3, activation='softmax')(x)

model = Model(sequence_input, preds)

#Optimizer
#sgd=K.SGD(learning_rate=1e-5, momentum=0.5)

#Model Compilation
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['acc'])

#Model Fitting
# # model.fit(X_train, y_train, validation_data=(X_test, y_test),
#           epochs=10, batch_size=32)

Model Summary

In [ ]:
model.summary()

Prediction

In [ ]:
pred=model.predict(X_test)


In [ ]:
print(pred)